In [ ]:
import os
import requests
from datetime import datetime, timedelta
from xml.dom import minidom
import sys
import threading
import pprint

BOE_URL = 'https://boe.es'
BOE_API_SUMARIO = 'https://boe.es/diario_boe/xml.php?id=BOE-S-'
START_DATE = datetime.strptime('20231001', '%Y%m%d')
PATH_DATA = os.path.join('boe', 'dias')
diff_1_day = timedelta(days=1)

# Scrappig BOE

## Funciones

In [ ]:
def make_dirs(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass

In [ ]:
def traer_documento(origen, destino):
    intentos = 0
    max_intentos = 5
    
    while intentos < max_intentos:
        if intentos != 0:
            import time
            time.sleep(5)
            print(f'Intento {intentos}')

        response = requests.get(origen, stream=True)
        if response.status_code == 200:
            print(f'Guardando en: {destino}')
            with open(destino, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            break
        else:
            intentos += 1

In [ ]:
def get_pdfs(current_date):
    print('current_date:', current_date.strftime('%Y-%m-%d'))

    fecha_anno, fecha_mes, fecha_dia = current_date.strftime('%Y %m %d').split()

    for dir_path in [os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia),
                    os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs')]:
        make_dirs(dir_path)

    fichero_sumario_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'index.xml')

    if os.path.exists(fichero_sumario_xml):
        os.remove(fichero_sumario_xml)

    print('Solicitando', f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")} --> {fichero_sumario_xml}')
    traer_documento(f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")}', fichero_sumario_xml)

    tamano_sumario_xml = os.path.getsize(fichero_sumario_xml)
    print('Recibidos:', tamano_sumario_xml, 'bytes')

    if tamano_sumario_xml < 10:
        print('ERROR: Sumario XML erróneo o incompleto')
        sys.exit(1)

    xml_sumario = minidom.parse(fichero_sumario_xml)

    if xml_sumario.documentElement.nodeName == 'error':
        os.remove(fichero_sumario_xml)
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs'))
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia))
        print('AVISO: No existen boletines para la current_date', current_date.strftime('%Y-%m-%d'))
    else:
        pdfs = xml_sumario.getElementsByTagName('urlPdf')
        print(f"{len(pdfs)} encontrados")
        for pdf in pdfs:
            fichero_pdf = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs', pdf.firstChild.nodeValue).replace(' ', '_').replace('/', '\\')[1:]
            fichero_pdf_tamano_xml = pdf.getAttribute('szBytes')
            if os.path.exists(fichero_pdf):
                print("fichero encontrado")
                if os.path.getsize(fichero_pdf) == int(fichero_pdf_tamano_xml):
                    continue
                else:
                    try:
                        os.remove(fichero_pdf)
                    except:
                        pass


            print('Solicitando', f'{BOE_URL}{pdf.firstChild.nodeValue} --> {fichero_pdf}')
            intentos = 0
            max_intentos = 5
            while intentos < max_intentos:
                if intentos != 0:
                    import time
                    time.sleep(5)
                    print(f'Intento {intentos}')

                traer_documento(f'{BOE_URL}{pdf.firstChild.nodeValue}', fichero_pdf)
                intentos += 1

                if os.path.getsize(fichero_pdf) == int(fichero_pdf_tamano_xml):
                    break

            if os.path.getsize(fichero_pdf) != int(fichero_pdf_tamano_xml):
                print('ERROR: El tamaño del fichero PDF descargado no coincide con el del XML del Sumario '
                    f'(Descargado: {os.getsize(fichero_pdf)} <> XML: {fichero_pdf_tamano_xml})')
                sys.exit(1)

In [ ]:
def get_xmls(current_date, semaphore=None):
    print('current_date:', current_date.strftime('%Y-%m-%d'))

    fecha_anno, fecha_mes, fecha_dia = current_date.strftime('%Y %m %d').split()

    for dir_path in [os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia),
                    os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls')]:
        make_dirs(dir_path)

    fichero_sumario_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'index.xml')

    if os.path.exists(fichero_sumario_xml):
        os.remove(fichero_sumario_xml)

    print('Solicitando', f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")} --> {fichero_sumario_xml}')
    traer_documento(f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")}', fichero_sumario_xml)

    tamano_sumario_xml = os.path.getsize(fichero_sumario_xml)
    print('Recibidos:', tamano_sumario_xml, 'bytes')

    if tamano_sumario_xml < 10:
        print('ERROR: Sumario XML erróneo o incompleto')
        sys.exit(1)

    xml_sumario = minidom.parse(fichero_sumario_xml)

    if xml_sumario.documentElement.nodeName == 'error':
        os.remove(fichero_sumario_xml)
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls'))
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia))
        print('AVISO: No existen boletines para la fecha ', current_date.strftime('%Y-%m-%d'))
    else:
        xmls = xml_sumario.getElementsByTagName('urlXml')
        for xml in xmls:
            fichero_xml = PATH_DATA + '\\' + str(fecha_anno) + '\\' + str(fecha_mes) + '\\' + str(fecha_dia) + '\\' + 'xmls' + '\\' + xml.firstChild.nodeValue.split('=')[-1] + '.xml'
            #fichero_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls', xml.firstChild.nodeValue.split('=')[-1] + '.xml').replace(' ', '_').replace('/', '\\')[1:]

            if os.path.exists(fichero_xml):
                print("fichero encontrado")
                continue


            print('Solicitando', f'{BOE_URL}{xml.firstChild.nodeValue} --> {fichero_xml}')
            intentos = 0
            max_intentos = 5
            while intentos < max_intentos:
                if intentos != 0:
                    import time
                    time.sleep(5)
                    print(f'Intento {intentos}')

                traer_documento(f'{BOE_URL}{xml.firstChild.nodeValue}', fichero_xml)
                intentos += 1

                if os.path.getsize(fichero_xml):
                    break

            if not os.path.getsize(fichero_xml):
                print('ERROR: El tamaño del fichero xml descargado no coincide con el del XML del Sumario '
                    f'(Descargado: {os.getsize(fichero_xml)} <> XML: {fichero_xml_tamano_xml})')
                sys.exit(1)

    if semaphore is not None:
        semaphore.release()

In [ ]:
current_date = START_DATE
hoy = datetime.now()
while current_date <= hoy and 0:
    get_xmls(current_date)
    get_pdfs(current_date)
    current_date += diff_1_day

In [ ]:
n_hilos = 6
semaphore = threading.Semaphore(n_hilos)
dates =  [START_DATE + timedelta(days=i) for i in range((datetime.now() - START_DATE).days + 1)]
hilos = []

print(dates)

for date in dates:
    semaphore.acquire()

    print('current_date:', date.strftime('%Y-%m-%d'))
    
    hilo = threading.Thread(target=get_xmls, args=(date, semaphore))
    #threading.Thread(target=get_pdfs, args=(date,)).start()
    hilos.append(hilo)
    hilo.start()

for hilo in hilos:
    hilo.join()


# Almacenamieto de datos

In [ ]:
def proceso_sumario(path_xml):
    data = {}
    xml_sumario = minidom.parse(path_xml)

    anno, mes, dia = path_xml.split('\\')[2:-2]
    file_name = path_xml.split('\\')[-1].split('.')[0]
    date = path_xml.split('\\')[-1].split('.')[0].split('-')[-2]

    data['index'] = file_name.replace(date, f'{anno}{mes}{dia}')

    aux = {}
    documento = xml_sumario.documentElement
    i = 0
    for element in documento.getElementsByTagName('*'):
        #print("Elemento:", element.tagName)
        if element.tagName == 'p': element.tagName = element.getAttribute('class') + '_' + str(i); i += 1
        data[element.tagName] = {}
        for attr_name, attr_value in element.attributes.items():
            data[element.tagName][attr_name] = attr_value
            #print(f"Atributo: {attr_name} - Valor: {attr_value}")
        if element.firstChild and element.firstChild.nodeType == element.TEXT_NODE:
            data[element.tagName]['Contenido'] = element.firstChild.data
            #print("Contenido:", element.firstChild.data)

    data_to_save = {}
    keys_to_save = ['identificador', 'index', 'origen_legislativo', 'departamento', 'titulo', 'fecha_publicacion']
    #keys_to_save = keys_to_save.extend([i for i in data.keys() if i.startswith('parrafo')])
    keys_to_save.extend([e for i, e in enumerate(data.keys()) if e.startswith('parrafo') or e.startswith('articulo')])
    for key in keys_to_save:
        try:
            if key == 'index':
                data_to_save[key] = data[key]
            elif key.startswith('parrafo'):
                data_to_save[key] = data[key]['Contenido'].replace('\xa0', ' ').replace('\u2003', ' ')
            else:
                data_to_save[key] = data[key]['Contenido'].replace('\xa0', ' ').replace('\u2003', ' ')
        except Exception as e:
            pass
            #print(f"{path_xml} excepcion: {e}")
    
    return data_to_save

In [ ]:
def process_data():
    data = {}
    i = 0
    #itero por las carpetas de boe
    for anio in os.listdir('boe/dias'):
        #itero por los archivos de cada carpeta
        for mes in os.listdir(f'boe/dias/{anio}'):
            #si el archivo es un pdf
            for dia in os.listdir(f'boe/dias/{anio}/{mes}'):
                path_xmls = os.path.join('boe', 'dias', anio, mes, dia, 'xmls')
                print(path_xmls)
                for xml in os.listdir(path_xmls):
                    path_xml = os.path.join(path_xmls, xml)
                    file_data = proceso_sumario(path_xml)
                    data[file_data['identificador']] = file_data
                    i += 1
                    #if i > 10: break
    
    return data

In [ ]:
def get_embeddings(input_text, model, tokenizer, max_length=512):
    enc = tokenizer(input_text, return_tensors='pt', add_special_tokens=True)
    output = model.encoder(
        input_ids=enc['input_ids'],
        attention_mask=enc['attention_mask'],
        return_dict=True
        )
    return output.last_hidden_state.tolist()[0][0]

In [ ]:
data = process_data()
data['BOE-A-2023-20397']

In [ ]:
#!pip install elasticsearch sentencepiece

In [ ]:
from elasticsearch import Elasticsearch
import json

#client = Elasticsearch("http://elasticsearch:9200")
client = Elasticsearch("http://localhost:9200")

## Crear index

In [ ]:
config = {
    "mappings": {
        "properties": {
           "text": {"type": "text", "similarity": "BM25"},
            "embeddings": {
                "type": "dense_vector",
                "dims": 512,
                "index": True,
                "similarity": "cosine"
            },
            "doc_id": {"type": "keyword"},
            "fecha_publicacion": {"type": "date"}
        }
    },
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    }
}

In [ ]:
import torch
from transformers import T5Model, T5Tokenizer

model = T5Model.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [ ]:
my_index = "boe"
try:
    client.indices.create(
        index = my_index,
        settings = config["settings"],
        mappings = config["mappings"]
    )
except Exception as error:
    print("Error:", error)

In [ ]:
for key in data.keys():
    print(key)
    fecha = datetime.strptime(data[key]['fecha_publicacion'], '%Y%m%d')
    id = key
    document = data[key]

    keys_to_save = [key for key in document.keys() if key.startswith('parrafo') or key.startswith('articulo')]
    for key_to_save in keys_to_save:
        try:
            client.index(
                index = my_index,
                document = {
                    "fecha_publicacion": fecha,
                    "doc_id": document['identificador'],
                    "text": document[key_to_save],
                    "embeddings": get_embeddings(document[key_to_save], model, tokenizer)
                },
            )
        except Exception as e:
            print(e)
            print(f"len: {len(document)}")
            print(fecha)
            print(id)
            print(document)
            break

In [ ]:
my_query = {
    "match_all": {}
}

res = client.search(index=my_index, body={"query": my_query}, size=100)
pprint.pprint(res['hits'])